In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

print("Versión de TensorFlow:", tf.__version__)

# ==========================================
# 1. IMPORTAR Y CARGAR EL DATASET
# ==========================================
# Fashion MNIST contiene 70,000 imágenes en escala de grises de 10 categorías.
# Las imágenes son arrays de NumPy de 28x28 píxeles, con valores de 0 a 255.
fashion_mnist = tf.keras.datasets.fashion_mnist

# load_data() descarga los datos y los divide en entrenamiento y prueba.
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Nombres de las clases (el dataset solo tiene etiquetas numéricas 0-9)
class_names = ['Camiseta/Top', 'Pantalón', 'Jersey', 'Vestido', 'Abrigo',
               'Sandalia', 'Camisa', 'Zapatilla', 'Bolso', 'Bota']

print(f"Set de Entrenamiento: {train_images.shape} (60,000 imágenes de 28x28)")
print(f"Set de Prueba: {test_images.shape} (10,000 imágenes)")

# ==========================================
# 2. PREPROCESAMIENTO DE DATOS
# ==========================================
# Escalamos los valores de píxeles de 0-255 a un rango de 0-1.
# Esto es crucial para que la red neuronal converja eficientemente.
train_images = train_images / 255.0
test_images = test_images / 255.0

# Visualización opcional de las primeras 5 imágenes
plt.figure(figsize=(10,2))
for i in range(5):
    plt.subplot(1,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

# ==========================================
# 3. CONSTRUCCIÓN DEL MODELO
# ==========================================
# Usamos keras.Sequential para apilar capas linealmente.
model = tf.keras.Sequential([
    # Capa 1: Aplana la imagen 2D (28x28) a un vector 1D (784 píxeles).
    # No tiene parámetros que aprender, solo reformatea datos.
    tf.keras.layers.Flatten(input_shape=(28, 28)),

    # Capa 2: Capa oculta densamente conectada con 128 neuronas.
    # Usa función de activación ReLU (Rectified Linear Unit).
    tf.keras.layers.Dense(128, activation='relu'),

    # Capa 3: Capa de salida con 10 neuronas (una por clase).
    # Usa softmax para devolver un array de 10 probabilidades que suman 1.
    tf.keras.layers.Dense(10, activation='softmax')
])

# ==========================================
# 4. COMPILACIÓN DEL MODELO
# ==========================================
model.compile(
    optimizer='adam',  # Optimizador: Cómo se actualiza el modelo basado en los datos y la pérdida.
    loss='sparse_categorical_crossentropy', # Función de pérdida: Mide qué tan exacto es el modelo.
    metrics=['accuracy'] # Métrica: Usamos exactitud (fracción de imágenes correctamente clasificadas).
)

# ==========================================
# 5. ENTRENAMIENTO DEL MODELO
# ==========================================
print("\n--- Iniciando Entrenamiento ---")
# fit() "ajusta" el modelo a los datos de entrenamiento.
# epochs=10: El modelo verá todo el set de entrenamiento 10 veces.
history = model.fit(train_images, train_labels, epochs=10)

# ==========================================
# 6. EVALUACIÓN Y PREDICCIÓN
# ==========================================
print("\n--- Evaluando en Set de Prueba ---")
# evaluate() comprueba el modelo con datos que NO ha visto nunca (test set).
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print(f'\nPrecisión en el set de prueba: {test_acc*100:.2f}%')

# Hacer predicciones sobre el set de prueba
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = model.predict(test_images)

# Verificar la predicción de la primera imagen
pred_idx = np.argmax(predictions[0])
real_idx = test_labels[0]

print(f"\nEjemplo de predicción (Imagen 0):")
print(f"Predicción: {class_names[pred_idx]} (Confianza: {np.max(predictions[0]):.4f})")
print(f"Realidad:   {class_names[real_idx]}")